In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn import metrics as skmetrics
from sklearn.preprocessing import StandardScaler

import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as Data



In [0]:
path = "/content/drive/My Drive/Colab Notebooks/Ethics/"
def save_obj(obj, name ):
    with open(path+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [0]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

Split Audult Dataset into training and test data:

In [0]:
# ADULT DATASET (saved features as for a reason aif360 was not working anymore)

metadata = load_obj('adult_data')
features = load_obj('features')
labels = load_obj('labels')
data = np.concatenate((features,labels),axis=1)


def split_data(data, split_point):
    assert split_point < 1 and split_point > 0
    np.random.shuffle(data)
    s = split_point
    N = int(data.shape[0] * s)
    X_train, y_train = data[:N][:,:-1], data[:N][:,-1]
    X_test, y_test = data[N:][:,:-1], data[N:][:,-1]

    return X_train, y_train , X_test, y_test

def preprocess(X_train, X_test):

    scale_orig = StandardScaler()
    x_train = scale_orig.fit_transform(X_train)
    x_test = scale_orig.transform(X_test)

    return x_train, x_test



Create a Logistic Regression class with pytorch:

In [0]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

Reweighing:

In [0]:
def reweighing(feat, lab, sens_class = 'sex'):
    if sens_class == 'sex':
        # Sex labels
        features = feat[:,1]
    elif sens_class == 'race':
        features = feat[:,0]
    else:
        raise 'Not valid class name'
    # Income Labels
    WEIGHTS = {}
    # W_(y,a) = count(Y=y) * count(A=a) / (count(Y=y, A=a) * N)
    # In total 2 unique weights
    for Y in range(2):
        WEIGHTS[Y] = {}
        for A in range(2):
            NY = sum(lab.ravel() == Y)
            NA = sum((features == A))
            NYA = sum((lab.ravel() == Y) & (features == A))
            WEIGHTS[Y][A] = {}
            WEIGHTS[Y][A] = (NY * NA / (NYA * lab.shape[0]))

    W = torch.zeros(lab.shape[0])
    for i in range(len(lab)):
        W[i] = WEIGHTS[lab[i]][features[i]]

    return W


In [0]:
GPU = True
device_idx = 0
if GPU:
    device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
    
BATCH_SIZE = 32
learning_rate = 0.0001


if device == 0:
    num_workers = 2
else:
    num_workers = 0


def get_loader(x_train, y_train, W):
    train_dataset = Data.TensorDataset(torch.tensor(x_train).float(), 
                                      torch.Tensor(y_train).float(), 
                                      W.float())
    
    return Data.DataLoader(dataset=train_dataset,
                           batch_size=BATCH_SIZE, 
                           shuffle=True, num_workers=num_workers)


In [0]:
# Split data 
X_train, y_train , X_test, y_test = split_data(data, 0.6)
# Preprocess data
x_train, x_test = preprocess(X_train, X_test)
# Get weights
W = reweighing(X_train, y_train)
# 
loader_train = get_loader(x_train, y_train, W)

In [123]:
reg_lambda = 48
num_splits = 10
male_tpr = []
male_tnr = []
female_tpr = []
female_tnr = []
ACC = []
TPR, TNR = [], []
for split in range(num_splits):
    # Split data 
    X_train, y_train , X_test, y_test = split_data(data, 0.7)
    # Preprocess data
    x_train, x_test = preprocess(X_train, X_test)
    # Get weights
    W = reweighing(X_train, y_train)
    # 
    loader_train = get_loader(x_train, y_train, W)
    model = LogisticRegression_torch(x_train.shape[1], 1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

    for epoch in range(epochs):
        train_loss = 0.0
        for i, (x, y, w) in enumerate(loader_train):
        # Converting inputs and labels to Variable

            inputs = Variable(x.to(device))
            labels = Variable(y.to(device))

            # Clear gradient buffers because we don't want any gradient 
            # from previous epoch to carry forward, dont want to cummulate gradients
            optimizer.zero_grad()

            # get output from the model, given the inputs
            outputs = model(inputs)

            # Regularization
            reg = 0
            for param in model.parameters():
                reg += 0.5 * (param ** 2).mean()
            
            # criterion
            criterion = torch.nn.BCELoss(weight=w, reduction='sum')
            # get loss for the predicted output
            loss = criterion(outputs.reshape(outputs.shape[0]), labels) + \
                reg_lambda * reg
            
                
            train_loss += loss.item()
            
            # get gradients w.r.t to parameters
            loss.backward()
            
            # update parameters
            optimizer.step()
        

        print('epoch [{}/{}], Training loss:{:.6f}'.format(
            epoch + 1, 
            epochs, 
            train_loss / len(loader_train.dataset)))
        
    with torch.no_grad():
        model.eval()
        out = model(Variable(torch.Tensor(x_test).to(device))).detach().cpu()
        pred = (out >= 0.5).int().numpy().squeeze()
        accuracy = sum((y_test == pred))/len(y_test)
     
    # Men 
    male = y_test[((X_test[:,1] == 1.0))]
    male_pred = pred[((X_test[:,1] == 1.0))]
    male_proba = out[((X_test[:,1] == 1.0))]

    # Female
    female = y_test[((X_test[:,1] == 0.0))]
    female_pred = pred[((X_test[:,1] == 0.0))]
    female_proba = out[((X_test[:,1] == 0.0))]

    ACC.append(accuracy)

    # GET TPR and TNR for each unique group
    tn, fp, fn, tp = CM(male, male_pred).ravel()
    male_tpr.append(tp/(tp+fn)) 
    male_tnr.append(tn/(tn+fp))
    


    tn, fp, fn, tp = CM(female, female_pred).ravel()
    female_tpr.append(tp/(tp+fn)) 
    female_tnr.append(tn/(tn+fp))

    TPR.append(abs(male_tpr[split] - female_tpr[split]))
    TNR.append(abs(male_tnr[split] - female_tnr[split]))

print('\nAccuracy:')
print('Mean:', np.mean(ACC), 'S.D.:', np.std(ACC))
print('\nMale:')
print('Mean TPR:', np.mean(male_tpr), 'S.D.:', np.std(male_tpr),
      'Mean TNR:', np.mean(male_tnr), 'S.D.:', np.std(male_tnr))
print('\nFemale:')
print('Mean TPR:', np.mean(female_tpr), 'S.D.:', np.std(female_tpr),
      'Mean TNR:', np.mean(female_tnr), 'S.D.:', np.std(female_tnr))
print('\n|TPR_m - TPR_f|:')
print('Mean :', np.mean(TPR), 'S.D.:', np.std(TPR))
print('\n|TNR_m - TNR_f|:')
print('Mean :', np.mean(TNR), 'S.D.:', np.std(TNR))


epoch [1/4], Training loss:0.636081
epoch [2/4], Training loss:0.617128
epoch [3/4], Training loss:0.615116
epoch [4/4], Training loss:0.614794
epoch [1/4], Training loss:0.656065
epoch [2/4], Training loss:0.618991
epoch [3/4], Training loss:0.616132
epoch [4/4], Training loss:0.615814
epoch [1/4], Training loss:0.637988
epoch [2/4], Training loss:0.619231
epoch [3/4], Training loss:0.617285
epoch [4/4], Training loss:0.616902
epoch [1/4], Training loss:0.669477
epoch [2/4], Training loss:0.618132
epoch [3/4], Training loss:0.615009
epoch [4/4], Training loss:0.614722
epoch [1/4], Training loss:0.656785
epoch [2/4], Training loss:0.619256
epoch [3/4], Training loss:0.616145
epoch [4/4], Training loss:0.615804
epoch [1/4], Training loss:0.680877
epoch [2/4], Training loss:0.619766
epoch [3/4], Training loss:0.615284
epoch [4/4], Training loss:0.614762
epoch [1/4], Training loss:0.653787
epoch [2/4], Training loss:0.619119
epoch [3/4], Training loss:0.616107
epoch [4/4], Training loss:0